In [1]:
!pip install news-please

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.9/272.9 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.4/265.4 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 K

In [2]:
import csv
import pandas as pd
import pickle
from itertools import accumulate, chain, repeat, tee
import difflib
import matplotlib
from newsplease import NewsPlease

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
filepath = '/content/drive/MyDrive/Project/VIP'

data = pd.read_csv('/content/drive/MyDrive/Project/VIP/allsides-content-f (4).csv')
    
data.head()

,Date,Description,Source_Name,Source_Bias,Source_Headline,Source_Link,Image_link
0,2018-03-08T15:34:58-08:00,The people who like the sort of tariffs that P...,OPINION: Trump’s risky call for protests,Left,https://www.allsides.com/news/2018-03-08-1534/...,http://money.cnn.com/2018/03/05/news/economy/t...,https://i2.cdn.turner.com/money/dam/assets/180...
1,2020-05-28T07:02:58-07:00,President Donald Trump is preparing to sign an...,OPINION: Trump’s risky call for protests,Left,https://www.allsides.com/news/2020-05-28-0702/...,https://apnews.com/fc30f9ebdff9d3d33a870e7374f...,https://storage.googleapis.com/afs-prod/media/...
2,2019-12-12T06:52:09-08:00,The Justice Department’s top watchdog on Wedne...,OPINION: Donald Trump has committed a lot of s...,Lean Left,https://www.allsides.com/news/2019-12-12-0651/...,https://www.politico.com/news/2019/12/11/horow...,https://cf-images.us-east-1.prod.boltdns.net/v...
3,2018-07-12T12:49:56-07:00,President Trump kept everyone guessing to the ...,Trump Indictment Could Be a 2024 Cash Cow,Center,https://www.allsides.com/news/2018-07-12-1249/...,https://www.wsj.com/articles/kavanaugh-for-the...,https://s.wsj.net/img/meta/wsj-social-share.png
4,2019-10-17T06:20:50-07:00,As President Donald Trump continues to fill hi...,Police surround NY Courthouse and DC Capitol i...,Center,https://www.allsides.com/news/2019-10-17-0620/...,https://abcnews.go.com/Politics/exclusive-hidi...,https://s.abcnews.com/images/US/Biden-intervie...


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             422 non-null    object
 1   Description      424 non-null    object
 2   Source_Name      424 non-null    object
 3   Source_Bias      422 non-null    object
 4   Source_Headline  424 non-null    object
 5   Source_Link      424 non-null    object
 6   Image_link       424 non-null    object
dtypes: object(7)
memory usage: 23.3+ KB


In [6]:
data['Source_Link'].values.tolist()

['http://money.cnn.com/2018/03/05/news/economy/trump-steel-aluminum-tariffs-support/index.html',
 'https://apnews.com/fc30f9ebdff9d3d33a870e7374f8ffa6',
 'https://www.politico.com/news/2019/12/11/horowitz-barr-trump-russia-probe-082448',
 'https://www.wsj.com/articles/kavanaugh-for-the-court-1531190695',
 'https://abcnews.go.com/Politics/exclusive-hiding-plain-sight-hunter-biden-defends-foreign/story?id=66275416',
 'https://nbcnews.com/tech/tech-news/trump-has-made-twitter-his-megaphone-fact-check-won-t-n1215421',
 'https://dailycaller.com/2019/03/25/media-russia-collusion-mueller-report-fake-news/',
 'https://www.washingtonexaminer.com/policy/defense-national-security/game-on-after-trump-declares-peace-talks-dead-us-commander-takes-aim-at-taliban',
 'https://www.foxnews.com/media/furor-over-trump-pardons-as-frustrated-barr-weighs-quitting',
 'https://www.npr.org/2020/01/27/799370370/what-to-watch-for-as-trumps-legal-team-resumes-its-impeachment-defense',
 'https://www.cnn.com/2019/09/

In [7]:
news_links = [link for link in data['Source_Link']]
news_links

['http://money.cnn.com/2018/03/05/news/economy/trump-steel-aluminum-tariffs-support/index.html',
 'https://apnews.com/fc30f9ebdff9d3d33a870e7374f8ffa6',
 'https://www.politico.com/news/2019/12/11/horowitz-barr-trump-russia-probe-082448',
 'https://www.wsj.com/articles/kavanaugh-for-the-court-1531190695',
 'https://abcnews.go.com/Politics/exclusive-hiding-plain-sight-hunter-biden-defends-foreign/story?id=66275416',
 'https://nbcnews.com/tech/tech-news/trump-has-made-twitter-his-megaphone-fact-check-won-t-n1215421',
 'https://dailycaller.com/2019/03/25/media-russia-collusion-mueller-report-fake-news/',
 'https://www.washingtonexaminer.com/policy/defense-national-security/game-on-after-trump-declares-peace-talks-dead-us-commander-takes-aim-at-taliban',
 'https://www.foxnews.com/media/furor-over-trump-pardons-as-frustrated-barr-weighs-quitting',
 'https://www.npr.org/2020/01/27/799370370/what-to-watch-for-as-trumps-legal-team-resumes-its-impeachment-defense',
 'https://www.cnn.com/2019/09/

Next, we are going to compile the list of URLs into chunks. To do this, we're going to create a function that accepts two arguments: the list of URLs and the number of chunks that we want to create.

Assuming that the number of chucks is greater than zero, the function is going to measure the size of the list and divide it by the number of chunks we are trying to create.

In [14]:
import math
def chuck(xs, n):
    assert n > 0
    L = len(xs)
    s, r = divmod(L, n)
    widths = chain(repeat(s+1, r), repeat(s, n-r))
    offsets = accumulate(chain((0,), widths))
    b, e = tee(offsets)
    next(e)
    return [xs[s] for s in map(slice, b, e)]


batch = chuck(news_links, 270)

batch[3:7]

[['https://dailycaller.com/2019/03/25/media-russia-collusion-mueller-report-fake-news/',
  'https://www.washingtonexaminer.com/policy/defense-national-security/game-on-after-trump-declares-peace-talks-dead-us-commander-takes-aim-at-taliban'],
 ['https://www.foxnews.com/media/furor-over-trump-pardons-as-frustrated-barr-weighs-quitting',
  'https://www.npr.org/2020/01/27/799370370/what-to-watch-for-as-trumps-legal-team-resumes-its-impeachment-defense'],
 ['https://www.cnn.com/2019/09/08/politics/michael-waltz-taliban-9-11-pompeo-cnntv/index.html',
  'https://abridgenews.com/topic/trump-cancels-taliban-talks'],
 ['http://www.csmonitor.com/USA/Politics/2016/0824/Trump-Clinton-Obama-How-do-their-immigration-plans-stack-up',
  'https://www.vox.com/2020/2/6/21125403/senate-trump-acquitted-not-democratic-18-million-53-percent-malapportionment']]

In [15]:
batch

[['http://money.cnn.com/2018/03/05/news/economy/trump-steel-aluminum-tariffs-support/index.html',
  'https://apnews.com/fc30f9ebdff9d3d33a870e7374f8ffa6'],
 ['https://www.politico.com/news/2019/12/11/horowitz-barr-trump-russia-probe-082448',
  'https://www.wsj.com/articles/kavanaugh-for-the-court-1531190695'],
 ['https://abcnews.go.com/Politics/exclusive-hiding-plain-sight-hunter-biden-defends-foreign/story?id=66275416',
  'https://nbcnews.com/tech/tech-news/trump-has-made-twitter-his-megaphone-fact-check-won-t-n1215421'],
 ['https://dailycaller.com/2019/03/25/media-russia-collusion-mueller-report-fake-news/',
  'https://www.washingtonexaminer.com/policy/defense-national-security/game-on-after-trump-declares-peace-talks-dead-us-commander-takes-aim-at-taliban'],
 ['https://www.foxnews.com/media/furor-over-trump-pardons-as-frustrated-barr-weighs-quitting',
  'https://www.npr.org/2020/01/27/799370370/what-to-watch-for-as-trumps-legal-team-resumes-its-impeachment-defense'],
 ['https://www.

In [16]:
def article_crawler():
    # crawler
    n = 0
    for i in range(0, len(batch)):
        try:
            slice = batch[i]
            # print slice
            slice_name = str(i) + '-NewsPlease-articleCrawl.p'
            article_information = NewsPlease.from_urls(slice)
            pickle.dump(article_information, open(slice_name, 'wb'))
            n += 1
        except:
            continue
            
article_crawler()

ERROR:newsplease.crawler.simple_crawler:not a 200 response: 400
ERROR:newsplease.crawler.simple_crawler:connection/timeout error: https://abridgenews.com/topic/trump-cancels-taliban-talks HTTPSConnectionPool(host='abridgenews.com', port=443): Max retries exceeded with url: /topic/trump-cancels-taliban-talks (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fad9ec384f0>: Failed to establish a new connection: [Errno 110] Connection timed out'))
ERROR:newsplease.crawler.simple_crawler:connection/timeout error: http://www.bigstory.ap.org/article/44d191fc22744697a65bab92d5c64d24/trump-outline-economic-plan-he-seeks-reverse-slide HTTPConnectionPool(host='www.bigstory.ap.org', port=80): Max retries exceeded with url: /article/44d191fc22744697a65bab92d5c64d24/trump-outline-economic-plan-he-seeks-reverse-slide (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fad9da27700>: Failed to establish a new connection: [Errno -2] Name or service

In [17]:
# helper function
def make_unique(url_list):
   # Not order preserving    
   unique = set(url_list)
   return list(unique)

def check_data(filepath):
    scraped = []
    not_scraped = []

    for i in range(0, (math.ceil(len(news_links)/2))):
        try:
            #file_path = filepath+"/crawl/"
            open_crawl = pickle.load(open(str(i) + "-NewsPlease-articleCrawl.p", "rb"))
            for url in open_crawl:
                text = open_crawl[str(url)].maintext
                if text == None:
                    not_scraped.append(url)
                else:
                    scraped.append(url)
        except FileNotFoundError:
            continue

    scraped = make_unique(scraped)
    return scraped, not_scraped
            
success, fail = check_data(filepath)

# analyze data collection success
def percentage(part, whole):
    percent = 100 * float(part)/float(whole)
    format = "{0:.2f}".format(percent)
    return format+'%'

print("The extraction process yielded " 
      + str(len(success)) + " articles, or "
      + percentage(len(success),len(news_links)) 
      + " of the total.")

The extraction process yielded 321 articles, or 75.71% of the total.


In [18]:
def get_data():
    news_dict = {}

    remove_list = ['www.', 'www1.' '.com', '.gov', '.org', 'beta.', ',eu',
                   '.co.uk', 'europe', 'gma', 'blogs', 'in.', 'm.',
                   'eclipse2017.', 'money', 'insider', 'news.', 'finance.'
                   'www1.']

    for i in range(0, 220):
        try:
            #file_path = filepath + "/crawl/"
            open_crawl = pickle.load(open(str(i) + "-NewsPlease-articleCrawl.p", "rb"))

            for url in open_crawl:
                text = open_crawl[str(url)].maintext
                if text != None:
                    title = open_crawl[str(url)].title
                    authors = ', '.join(open_crawl[str(url)].authors)
                    source = open_crawl[str(url)].source_domain

                    for seq in remove_list:
                        if seq in source:
                            source = source.replace(seq, "")

                    date = open_crawl[str(url)].date_publish

                    news_dict[str(url)] = [source, title, authors, date, text]

        except FileNotFoundError:
            continue

    return news_dict


all_news = get_data()
print(all_news)

{'https://apnews.com/fc30f9ebdff9d3d33a870e7374f8ffa6': ['apcom', 'Trump escalates war on Twitter, social media protections', 'Zeke Miller', datetime.datetime(2021, 4, 20, 12, 45, 50), 'FILE - In this Oct. 11, 2017, file photo, MSNBC television anchor Joe Scarborough takes questions from an audience at forum at the John F. Kennedy School of Government, on the campus of Harvard University, in Cambridge, Mass. The husband of a woman who died accidentally in an office of then-GOP Rep. Joe Scarborough two decades ago is demanding that Twitter remove President Donald Trump’s tweets suggesting Scarborough murdered her. (AP Photo/Steven Senne, File)\nFILE - In this Oct. 11, 2017, file photo, MSNBC television anchor Joe Scarborough takes questions from an audience at forum at the John F. Kennedy School of Government, on the campus of Harvard University, in Cambridge, Mass. The husband of a woman who died accidentally in an office of then-GOP Rep. Joe Scarborough two decades ago is demanding th

In [19]:
def get_sources(all_news):
    news_sources = {}
    for article in all_news:
        source = all_news[article][0]
        if source not in news_sources:
            news_sources[source] = 1
        else:
            news_sources[source] += 1
    return news_sources

check_sources = get_sources(all_news)

check_sources

{'apcom': 9,
 '.cnn.com': 1,
 'wsj.com': 23,
 'politico.com': 10,
 'abcgo.com': 3,
 'nbccom': 6,
 'dailycaller.com': 6,
 'washingtonexaminer.com': 15,
 'npr': 9,
 'foxcom': 16,
 'cnn.com': 9,
 'csmonitor.com': 9,
 'vox.com': 4,
 'factcheck': 2,
 'business.com': 4,
 'reuters.com': 18,
 '.wsj.com': 1,
 'bigthink.com': 1,
 'washingtonpost.com': 27,
 'slate.com': 3,
 'theguardian.com': 6,
 'bbc.com': 6,
 'theepochtimes.com': 3,
 'usatoday.com': 17,
 'newsweek.com': 6,
 'nytimes.com': 2,
 'theatlantic.com': 2,
 'townhall.com': 9,
 'huffingtonpost.com': 4,
 'thehill.com': 13,
 'axios.com': 3,
 'motherjones.com': 2,
 'nypost.com': 9,
 'cbscom': 4,
 'salon.com': 2,
 'ibtimes.com': 1,
 'thefederalist.com': 7,
 'buzzfeedcom': 2,
 'mediamatters': 1,
 'dailywire.com': 4,
 'vanityfair.com': 2,
 'washingtontimes.com': 12,
 'reason.com': 3,
 'www1.cbn.com': 1,
 'forbes.com': 1,
 'bloomberg.com': 3,
 'huffpost.com': 1,
 'newyorker.com': 4,
 'nydailycom': 1,
 'msnbc.com': 1,
 'cnbc.com': 2,
 'cnet.com'

In [ ]:
print(all_news)

In [20]:
data['main_headline'] = data['Source_Link'].apply(lambda x: all_news[x][1] if x in all_news else None)
data['text'] = data['Source_Link'].apply(lambda x: all_news[x][4] if x in all_news else None)
data['author'] = data['Source_Link'].apply(lambda x: all_news[x][2] if x in all_news else None)

In [ ]:
data.head()

In [22]:
from urllib.parse import urlparse

def extract_domain(url):
    parsed_url = urlparse(url)
    domain = parsed_url.hostname.split('.')[-2]
    return domain

In [23]:
data['Source_Headline'] = data['Source_Link'].apply(extract_domain)
data = data.rename(columns={'Date': 'data', 'Description': 'description', 'Source_Name': 'source name', 'Source_Bias': 'bias', 
                            'Source_Headline': 'source', 'Source_Link': 'link'})
data.head()

,data,description,source name,bias,source,link,Image_link,main_headline,text,author
0,2018-03-08T15:34:58-08:00,The people who like the sort of tariffs that P...,OPINION: Trump’s risky call for protests,Left,cnn,http://money.cnn.com/2018/03/05/news/economy/t...,https://i2.cdn.turner.com/money/dam/assets/180...,"Who likes tariffs? Generally speaking, it's no...",The people who like the sort of tariffs that P...,Chris Isidore
1,2020-05-28T07:02:58-07:00,President Donald Trump is preparing to sign an...,OPINION: Trump’s risky call for protests,Left,apnews,https://apnews.com/fc30f9ebdff9d3d33a870e7374f...,https://storage.googleapis.com/afs-prod/media/...,"Trump escalates war on Twitter, social media p...","FILE - In this Oct. 11, 2017, file photo, MSNB...",Zeke Miller
2,2019-12-12T06:52:09-08:00,The Justice Department’s top watchdog on Wedne...,OPINION: Donald Trump has committed a lot of s...,Lean Left,politico,https://www.politico.com/news/2019/12/11/horow...,https://cf-images.us-east-1.prod.boltdns.net/v...,Horowitz pushes back at Barr over basis for Tr...,Graham was also critical of the FBI’s decision...,
3,2018-07-12T12:49:56-07:00,President Trump kept everyone guessing to the ...,Trump Indictment Could Be a 2024 Cash Cow,Center,wsj,https://www.wsj.com/articles/kavanaugh-for-the...,https://s.wsj.net/img/meta/wsj-social-share.png,Kavanaugh for the Court,Judge Brett Kavanaugh speaks after being nomin...,The Editorial Board
4,2019-10-17T06:20:50-07:00,As President Donald Trump continues to fill hi...,Police surround NY Courthouse and DC Capitol i...,Center,go,https://abcnews.go.com/Politics/exclusive-hidi...,https://s.abcnews.com/images/US/Biden-intervie...,Exclusive: 'I'm here': Hunter Biden hits back ...,As President Donald Trump continues to fill hi...,Abc News


In [29]:
data.dropna(inplace = True)
data.reset_index(drop = True)

,data,description,source name,bias,source,link,Image_link,main_headline,text,author
0,2018-03-08T15:34:58-08:00,The people who like the sort of tariffs that P...,OPINION: Trump’s risky call for protests,Left,cnn,http://money.cnn.com/2018/03/05/news/economy/t...,https://i2.cdn.turner.com/money/dam/assets/180...,"Who likes tariffs? Generally speaking, it's no...",The people who like the sort of tariffs that P...,Chris Isidore
1,2020-05-28T07:02:58-07:00,President Donald Trump is preparing to sign an...,OPINION: Trump’s risky call for protests,Left,apnews,https://apnews.com/fc30f9ebdff9d3d33a870e7374f...,https://storage.googleapis.com/afs-prod/media/...,"Trump escalates war on Twitter, social media p...","FILE - In this Oct. 11, 2017, file photo, MSNB...",Zeke Miller
2,2019-12-12T06:52:09-08:00,The Justice Department’s top watchdog on Wedne...,OPINION: Donald Trump has committed a lot of s...,Lean Left,politico,https://www.politico.com/news/2019/12/11/horow...,https://cf-images.us-east-1.prod.boltdns.net/v...,Horowitz pushes back at Barr over basis for Tr...,Graham was also critical of the FBI’s decision...,
3,2018-07-12T12:49:56-07:00,President Trump kept everyone guessing to the ...,Trump Indictment Could Be a 2024 Cash Cow,Center,wsj,https://www.wsj.com/articles/kavanaugh-for-the...,https://s.wsj.net/img/meta/wsj-social-share.png,Kavanaugh for the Court,Judge Brett Kavanaugh speaks after being nomin...,The Editorial Board
4,2019-10-17T06:20:50-07:00,As President Donald Trump continues to fill hi...,Police surround NY Courthouse and DC Capitol i...,Center,go,https://abcnews.go.com/Politics/exclusive-hidi...,https://s.abcnews.com/images/US/Biden-intervie...,Exclusive: 'I'm here': Hunter Biden hits back ...,As President Donald Trump continues to fill hi...,Abc News
...,...,...,...,...,...,...,...,...,...,...
322,2018-06-14T14:23:22-07:00,U.S. President Donald Trump said on Wednesday ...,House Passes Two Articles of Impeachment again...,Right,reuters,https://www.reuters.com/article/us-northkorea-...,https://static.reuters.com/resources/r/?m=02&a...,Trump says summit removed North Korean nuclear...,SEOUL/WASHINGTON (Reuters) - U.S. President Do...,"David Brunnstrom, Steve Holland"
323,2018-10-26T13:56:18-07:00,President Donald Trump on Wednesday implored p...,'Suicide march': Defiant Trump vows impeachmen...,Lean Right,usatoday,https://www.usatoday.com/story/news/politics/2...,https://www.gannett-cdn.com/presto/2018/10/24/...,Trump calls for civility during MAGA rally,"Trump tones down attacks, calls for civility a...","Christal Hayes, John Fritze"
324,2019-08-21T08:04:24-07:00,"In any ordinary year, Donald Trump's big win i...","President Donald Trump impeached by US House, ...",Lean Left,npr,http://www.npr.org/2016/02/23/467734676/why-ar...,https://media.npr.org/assets/img/2016/02/22/ge...,Why Aren't People Declaring The GOP Race Over ...,Why Aren't People Declaring The GOP Race Over ...,Mara Liasson
325,2019-07-19T07:11:44-07:00,go back,Putin Criticizes Democrats for Impeaching Trump,Center,thehill,https://thehill.com/homenews/media/453331-merr...,https://thehill.com/wp-content/uploads/sites/2...,Merriam-Webster appears to jab Brit Hume for c...,The official Twitter account for Merriam-Webst...,"Morgan Gstalter, Hours Ago, Pm Et, Days Ago, A..."


In [ ]:
data['author'].tolist()

In [ ]:
data.info()

Now we have more complete information. However, we still want to clean it up a bit. We're going to start by dropping all of the rows with missing stories (or mixed stories). After that, we're going to drop the stories that are less than 250 characters and greater than 20,000 characters.

Afterward, we'll save the results to a CSV for further analysis later on.

In [34]:
data = data.drop(data[data.text == 'None'].index)
data = data.drop(data[data.text == 'Mixed'].index)

data['text_len'] = data.apply(lambda row: len(row.text), axis=1)

data = data.drop(data[data.text_len > 20000].index)
data = data.drop(data[data.text_len < 250].index)

# write to csv
data.to_csv('news-corpus-df.csv', index=False)
data.head()


,data,description,source name,bias,source,link,Image_link,main_headline,text,author,text_len
0,2018-03-08T15:34:58-08:00,The people who like the sort of tariffs that P...,OPINION: Trump’s risky call for protests,Left,cnn,http://money.cnn.com/2018/03/05/news/economy/t...,https://i2.cdn.turner.com/money/dam/assets/180...,"Who likes tariffs? Generally speaking, it's no...",The people who like the sort of tariffs that P...,Chris Isidore,3013
1,2020-05-28T07:02:58-07:00,President Donald Trump is preparing to sign an...,OPINION: Trump’s risky call for protests,Left,apnews,https://apnews.com/fc30f9ebdff9d3d33a870e7374f...,https://storage.googleapis.com/afs-prod/media/...,"Trump escalates war on Twitter, social media p...","FILE - In this Oct. 11, 2017, file photo, MSNB...",Zeke Miller,8726
2,2019-12-12T06:52:09-08:00,The Justice Department’s top watchdog on Wedne...,OPINION: Donald Trump has committed a lot of s...,Lean Left,politico,https://www.politico.com/news/2019/12/11/horow...,https://cf-images.us-east-1.prod.boltdns.net/v...,Horowitz pushes back at Barr over basis for Tr...,Graham was also critical of the FBI’s decision...,,6234
3,2018-07-12T12:49:56-07:00,President Trump kept everyone guessing to the ...,Trump Indictment Could Be a 2024 Cash Cow,Center,wsj,https://www.wsj.com/articles/kavanaugh-for-the...,https://s.wsj.net/img/meta/wsj-social-share.png,Kavanaugh for the Court,Judge Brett Kavanaugh speaks after being nomin...,The Editorial Board,1062
4,2019-10-17T06:20:50-07:00,As President Donald Trump continues to fill hi...,Police surround NY Courthouse and DC Capitol i...,Center,go,https://abcnews.go.com/Politics/exclusive-hidi...,https://s.abcnews.com/images/US/Biden-intervie...,Exclusive: 'I'm here': Hunter Biden hits back ...,As President Donald Trump continues to fill hi...,Abc News,15428


In [35]:
bybias = data.groupby('bias')
bybias['text_len'].describe()

,count,mean,std,min,25%,50%,75%,max
bias,,,,,,,,
Center,69.0,5269.652174,3746.856713,398.0,2851.00,4403.0,7109.00,19419.0
Lean Left,100.0,5461.380000,3544.331499,252.0,3462.50,4630.0,6752.75,18648.0
Lean Right,31.0,5477.419355,3551.942584,320.0,3039.50,5486.0,7042.50,14472.0
Left,40.0,5865.950000,3390.754390,387.0,3646.25,5033.0,8455.75,13480.0
Right,78.0,5106.910256,3386.192200,340.0,2611.50,4617.0,6764.50,14784.0
